Dữ liệu đầu vào
- Bán từ 6 tháng trước trở lên (không tính tháng hiện tại) => 6 tháng mà trong đó có 3 tháng train thì quá ngắn => tăng lên 9 tháng
- Bán đều: Số tháng bán được hàng thực tế >= 6 tháng
- Đang bán trong tháng gần nhất M-1
- Bán top 5 theo gian hàng, chi nhánh, category lv2
- Số lượng bán trung bình tháng >= 50
  
Model
- Forecast 1 tháng
- Forecast 2 tháng
- Forecast 3 tháng

Test
- Các case có kết quả predict = 0 (115 case), test random 20 case
- Best param
- Forecast 1 tháng
- Forecast 2 tháng
- Forecast 3 tháng

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import calendar
from dateutil.relativedelta import relativedelta

from sqlalchemy import create_engine
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from itertools import product
from sklearn.metrics import mean_squared_error

import warnings
import time
from collections import defaultdict

warnings.simplefilter(action='ignore', category=Warning)

## Get raw data

In [2]:
df = pd.read_csv('test_deploy_model_05.07.csv')

## Transform & clean data

In [3]:
# Format lại cột txn_month
df['txn_month'] = pd.to_datetime(df['txn_month'], format='%Y-%m-%d')

df = df[['txn_month','concat_product_key', 'no_sold']]

In [4]:
df = df.set_index('txn_month')

In [5]:
all_products = df['concat_product_key'].unique()

## Test thử với n case

In [6]:
# Tạo danh sách product master key distinct
list_prod = list(df['concat_product_key'].unique())

# Lấy ra n giá trị trong danh sách trên
filter_list_prod = list_prod[0:20]

# Lọc ra các dữ liệu của các product đã chọn
df = df[df['concat_product_key'].isin(filter_list_prod)]

## Modelling

### Model function

In [6]:
# Khởi tạo hàm tính sử dụng ADFuler test để tính độ trễ i cho sai phân bậc 1 có p-value bé nhất => chuỗi dừng tương ứng
def checking_stationary(df, pred_n_month):
    diff = 4   # Đang mặc định diff = 4
    best_diff = None
    min_pvalue = float('inf')
    
    for i in range(pred_n_month, diff):
        diff_df = df['no_sold'] - df['no_sold'].shift(i) #Sai phân bậc 1 với độ trễ i
        res = adfuller(diff_df.dropna())
        p_value = res[1]
        if p_value < min_pvalue:
            min_pvalue = p_value
            best_diff = i
    return best_diff, min_pvalue # Trả về bậc sai phân và p-value tương ứng

In [7]:
# Khởi tạo hàm evaluate_sarima chạy model SARIMAX trả về mse - sai số giữa forecast và actual
def evaluate_sarima(params, shift_df, pred_n_month):
    order = params[:3]  # Non-seasonal parameters
    seasonal_order = params[3:]  # Seasonal parameters

    # Define the start and end dates for training and testing periods
    train_start_date = '2022-01-01'
    train_end_date = (datetime.now() - relativedelta(months = pred_n_month + 1)).strftime('%Y-%m-01')
    test_start_date = (datetime.now() - relativedelta(months = pred_n_month)).strftime('%Y-%m-01')

    # Split the dataset into training and testing sets
    train_set = shift_df.loc[train_start_date:train_end_date]
    test_set = shift_df.loc[test_start_date:]

    try:
        # Fit SARIMA model
        model = SARIMAX(train_set, order=order, seasonal_order=seasonal_order, enforce_stationarity=False) # Sử dụng tập train để train dữ liệu
        results = model.fit(disp=False)
        forecasts = results.forecast(steps = pred_n_month) # Và forecast ra các giá trị tương ứng với tập test

        # Calculate MSE as evaluation metric
        mse = mean_squared_error(test_set, forecasts) # Trung bình của tổng các bình phương sai số giữa các giá trị thực tế và giá trị dự đoán
        return mse #Trả về sai số của forecast và actual (mse)
    except:
        return None

In [8]:
# Khởi tạo hàm optimal_params trả về bộ param tốt nhất (có mse nhỏ nhất)
# => Sửu dụng hàm checking_stationary để tính độ trễ cho sai phân bậc 1 tối ưu nhất => chuỗi dừng tương ứng
# => Sửu dụng hàm evaluate_sarima lần lượt cho từng bộ param và chuỗi dừng => mse

def optimal_params(shift_df, pred_n_month):
    # Define range of parameters for grid search
    p = range(1, 4)  
    d = range(0, 2)
    q = range(1, 4)  
    P = range(1, 4)  
    D = range(0, 2)  
    Q = range(1, 4)  
    s = [6, 9, 12]   # Chu kỳ nửa năm (tháng)
    
    # Tạo tất cả các kết hợp tham số có thể có
    param_grid = product(p, d, q, P, D, Q, s)

    best_mse = float('inf')
    best_params = None
    
    for params in param_grid: # Vòng lặp cho từng bộ param
        mse = evaluate_sarima(params, shift_df, pred_n_month) # Gọi đến hàm evaluate_sarima đã khai báo ở trên và trả ra sai số của mỗi bộ
        if mse is not None and mse < best_mse:
          best_mse = mse
          best_params = params
    return best_params

In [9]:
# Khởi tạo hàm sarimax_model chạy model SARIMAX
# => Truyền vào hàm optimal_params ra bộ param tốt nhất
# => Train model với bộ param tốt nhất đó

def sarimax_model(prod, df, pred_n_month):
    best_diff = checking_stationary(df, pred_n_month)[0]
    
    shift_df = df['no_sold'] - df['no_sold'].shift(best_diff)
    shift_df = shift_df.dropna()
    best_params = optimal_params(shift_df, pred_n_month)
    
    # Fit ARIMA model on training data
    model = SARIMAX(shift_df, order = best_params[:3], seasonal_order= best_params[3:], enforce_stationarity=False)  # Example order, adjust as needed
    results = model.fit()
    
    # Generate predictions for test set
    forecast = results.forecast(steps=pred_n_month)

 # Create a dataframe which contains predicted values
    result = pd.concat([df[['no_sold']], forecast], axis = 1) # Nối thêm cột dự báo vào df cũ
    result.columns = ['original', 'forecast'] # Đặt lại tên mới cho 2 cột
    result['forecast'] = result['forecast'] + result['original'].shift(best_diff)
    result['concat_product_key'] = prod

    
    # Bỏ các dòng forecast = NA và thay thế các giá trị âm trong cột forecast bằng 0
    result = result.dropna(subset=['forecast'])
    result['forecast'] = result['forecast'].apply(lambda x: 0 if x < 0 else x)
    
    return result[['concat_product_key', 'forecast']], best_params

In [10]:
df

,concat_product_key,no_sold
txn_month,,
2023-09-01,126756293126251251263729005,96.0
2023-09-01,1205928431202216612017801934,34.0
2023-09-01,11233608411217791411232826101,0.0
2023-09-01,120621717120529491206852385,7940.0
2023-09-01,1288147061282888612812254730,0.0
...,...,...
2024-05-01,1174414471176338711713172062,94.0
2024-05-01,105165966105133610516331806,20.0
2024-05-01,137637118137623913728840563,65.0


In [14]:
prod = '120621717120529491206852385'
target_df = df[df['concat_product_key'] == prod]
target_df = target_df.sort_index()
pred_1m_df, best_params_1m = sarimax_model(prod, target_df, pred_n_month = 1)


C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zero

C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal 

C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal 

C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal 

C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal 

...

C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal 

C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal 

C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal 

C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
C:\Users\hieulm8\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal 

TypeError: 'NoneType' object is not subscriptable

In [15]:
prod = '120621717120529491206852385'
target_df = df[df['concat_product_key'] == prod]
target_df

,concat_product_key,no_sold
txn_month,,
2023-09-01,120621717120529491206852385,7940.0
2023-10-01,120621717120529491206852385,260.0
2023-11-01,120621717120529491206852385,120.0
2023-12-01,120621717120529491206852385,50.0
2024-01-01,120621717120529491206852385,0.0
2024-02-01,120621717120529491206852385,0.0
2024-03-01,120621717120529491206852385,0.0
2024-04-01,120621717120529491206852385,1080.0
2024-05-01,120621717120529491206852385,6600.0


### Running all merchant

In [11]:
# Khởi tạo df lưu kết quả predict và best params
result_1m_df = pd.DataFrame()
result_2m_df = pd.DataFrame()
result_3m_df = pd.DataFrame()
saved_best_params = pd.DataFrame()

# Biến đếm số lượng thành công và lỗi
success_count = 0
error_counts = defaultdict(int)

for index, prod in enumerate(all_products):
    start_time = time.time()  
    try:
        target_df = df[df['concat_product_key'] == prod]
        target_df = target_df.sort_index()
        
        pred_1m_df, best_params_1m = sarimax_model(prod, target_df, pred_n_month = 1)
        pred_2m_df, best_params_2m = sarimax_model(prod, target_df, pred_n_month = 2)
        pred_3m_df, best_params_3m = sarimax_model(prod, target_df, pred_n_month = 3)
        
        result_1m_df = pd.concat([result_1m_df, pred_1m_df], ignore_index=False)  # Mỗi vòng lặp nối thêm kết quả pred_1m_df vào result_1m_df
        result_2m_df = pd.concat([result_2m_df, pred_2m_df], ignore_index=False)  # Mỗi vòng lặp nối thêm kết quả pred_2m_df vào result_2m_df
        result_3m_df = pd.concat([result_3m_df, pred_3m_df], ignore_index=False)  # Mỗi vòng lặp nối thêm kết quả pred_3m_df vào result_3m_df

        prod_params = pd.DataFrame([{
                                      'concat_product_key': prod, 
                                      'best_params_1m':str(best_params_1m), 
                                      'best_params_2m':str(best_params_2m), 
                                      'best_params_3m':str(best_params_3m)
                                    }])
        
        saved_best_params = pd.concat([saved_best_params, prod_params], ignore_index=False)  # Mỗi vòng lặp nối thêm kết quả prod_params vào saved_best_params


        print("Finish concat_product_key {}. Time: {}".format(prod, time.time() - start_time))
        print(pred_1m_df, best_params_1m)
        print(pred_2m_df, best_params_2m)
        print(pred_3m_df, best_params_3m)
        
        # Tăng biến đếm thành công
        success_count += 1
        
    except TypeError as error:
        print("TypeError occurred for concat_product_key {}: {}. Time: {}".format(prod, error, time.time() - start_time))
        
        # Tăng biến đếm cho TypeError
        error_counts["TypeError"] += 1
        continue
        
    except Exception as error:
        print("Unexpected error occurred for concat_product_key {}: {}. Time: {}".format(prod, error, time.time() - start_time))
        
        # Tăng biến đếm cho lỗi không mong đợi
        error_counts["UnexpectedError"] += 1
        continue

# In kết quả đếm
print("Number of successful:", success_count)
print("Error counts by type:")
for error_type, count in error_counts.items():
    print(f"{error_type}: {count}")

TypeError occurred for concat_product_key 126756293126251251263729005: 'NoneType' object is not subscriptable. Time: 14.873107194900513
TypeError occurred for concat_product_key 1205928431202216612017801934: 'NoneType' object is not subscriptable. Time: 14.247993469238281
TypeError occurred for concat_product_key 11233608411217791411232826101: 'NoneType' object is not subscriptable. Time: 14.198325872421265
TypeError occurred for concat_product_key 120621717120529491206852385: 'NoneType' object is not subscriptable. Time: 13.619630098342896
TypeError occurred for concat_product_key 1288147061282888612812254730: 'NoneType' object is not subscriptable. Time: 13.9429030418396
TypeError occurred for concat_product_key 1288095281282455712811412641: 'NoneType' object is not subscriptable. Time: 14.456064939498901
TypeError occurred for concat_product_key 108237369108727810825012881: 'NoneType' object is not subscriptable. Time: 14.54102349281311
TypeError occurred for concat_product_key 1052

## Saving output

In [18]:
# Đổi tên cột tháng dự báo thành txn_month
result_1m_df.reset_index(inplace=True)
result_2m_df.reset_index(inplace=True)
result_3m_df.reset_index(inplace=True)

result_1m_df = result_1m_df.rename(columns={'index': 'txn_month'})
result_2m_df = result_2m_df.rename(columns={'index': 'txn_month'})
result_3m_df = result_3m_df.rename(columns={'index': 'txn_month'})

In [22]:
# Join lại với bảng ban đầu để lấy các cột khác
merged_1m_df = result_1m_df.merge(df_full, on='concat_product_key', how='left')
merged_2m_df = result_2m_df.merge(df_full, on='concat_product_key', how='left')
merged_3m_df = result_3m_df.merge(df_full, on='concat_product_key', how='left')
saved_best_params = saved_best_params.merge(df_full, on='concat_product_key', how='left')

In [24]:
# Thêm cột created_date là ngày chạy dữ liệu
merged_1m_df['created_date'] = datetime.now().strftime('%Y-%m-%d')
merged_2m_df['created_date'] = datetime.now().strftime('%Y-%m-%d')
merged_3m_df['created_date'] = datetime.now().strftime('%Y-%m-%d')
saved_best_params['created_date'] = datetime.now().strftime('%Y-%m-%d')

In [ ]:
# Save output
merged_1m_df.to_csv('predict_1m.csv')
merged_2m_df.to_csv('predict_2m.csv')
merged_3m_df.to_csv('predict_3m.csv')
saved_best_params.to_csv('best_params')